In [2]:
import pandas as pd
import numpy as np
import pymssql
import pyodbc

ImportError: libodbc.so.2: cannot open shared object file: No such file or directory

In [3]:
# connect with database
conn = pymssql.connect(server="20.198.233.3", user="PowerBI", password="Powerbi@12345", database="Cultive8Prod")